In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        #out = self.softmax(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.3456
Epoch [1/5], Step [200/600], Loss: 0.3313
Epoch [1/5], Step [300/600], Loss: 0.2336
Epoch [1/5], Step [400/600], Loss: 0.1440
Epoch [1/5], Step [500/600], Loss: 0.3188
Epoch [1/5], Step [600/600], Loss: 0.0820
Epoch [2/5], Step [100/600], Loss: 0.0959
Epoch [2/5], Step [200/600], Loss: 0.1439
Epoch [2/5], Step [300/600], Loss: 0.0734
Epoch [2/5], Step [400/600], Loss: 0.0505
Epoch [2/5], Step [500/600], Loss: 0.1090
Epoch [2/5], Step [600/600], Loss: 0.0722
Epoch [3/5], Step [100/600], Loss: 0.0555
Epoch [3/5], Step [200/600], Loss: 0.0686
Epoch [3/5], Step [300/600], Loss: 0.0487
Epoch [3/5], Step [400/600], Loss: 0.0528
Epoch [3/5], Step [500/600], Loss: 0.1659
Epoch [3/5], Step [600/600], Loss: 0.0856
Epoch [4/5], Step [100/600], Loss: 0.0601
Epoch [4/5], Step [200/600], Loss: 0.0796
Epoch [4/5], Step [300/600], Loss: 0.0646
Epoch [4/5], Step [400/600], Loss: 0.0637
Epoch [4/5], Step [500/600], Loss: 0.0588
Epoch [4/5], Step [600/600], Loss:

In [2]:
# This code loads model from the saved checkpoint file
modelB = NeuralNet(input_size, hidden_size, num_classes)
modelB.load_state_dict(torch.load('model.ckpt'),strict=False)

In [3]:
# This code grabs a random image and its associated label in the first batch (of 100 image-label pairs)
images, labels = next(iter(test_loader))
images = images.reshape(-1, 28,28)
example = images[30]
correct = labels[30]

In [4]:
# Visualization of the selected image
import matplotlib.pyplot as plt
plt.imshow(example,cmap="gray")
plt.show()

<Figure size 640x480 with 1 Axes>

In [5]:
# Flatten image into a vector that can be fed through the neural net
image_tensor = example.reshape(-1,28*28)
print(image_tensor.shape)

torch.Size([1, 784])


In [6]:
# Feedforward process of the one example
with torch.no_grad():
    outputs = modelB(image_tensor)
    print(outputs)
    _, predicted = torch.max(outputs.data,1)
    print(predicted[0])
    print(correct)

tensor([[-10.4232,  -6.6709,  -8.5338,  12.3886, -13.2222,   1.0764,
         -17.6158,  -7.0606,  -3.9162,  -1.2951]])
tensor(3)
tensor(3)
